In [314]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [298]:
df = pd.read_csv('train.csv')


In [299]:
df.outcome_type.unique()

array(['Return to Owner', 'Missing', 'Transfer', 'Adoption', 'Euthanasia',
       'Rto-Adopt', 'Relocate', 'Died', 'Disposal'], dtype=object)

In [300]:
df.head()

,animal_id_outcome,dob_year,dob_month,age_upon_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,...,age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,outcome_type
0,A006100,2007,7,10 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Stray,Neutered Male,...,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,12,2017,2017-12,Thursday,0,1.0,Return to Owner
1,A006100,2007,7,7 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,...,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,12,2014,2014-12,Saturday,16,2.0,Return to Owner
2,A006100,2007,7,6 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,...,6.0,"(5.0, 7.5]",NaN,3,2014,2014-03,Saturday,17,3.0,Return to Owner
3,A047759,2004,4,10 years,Dog,Dachshund,Tricolor,Normal,Owner Surrender,Neutered Male,...,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,4,2014,2014-04,Monday,15,1.0,Missing
4,A134067,1997,10,16 years,Dog,Shetland Sheepdog,Brown/White,Injured,Public Assist,Neutered Male,...,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,11,2013,2013-11,Saturday,11,1.0,Return to Owner


In [301]:
df.isnull().sum()
values = {'sex_upon_outcome': 'Unknown'}
df = df.fillna(value=values)
df.dtypes

animal_id_outcome              object
dob_year                        int64
dob_month                       int64
age_upon_intake                object
animal_type                    object
breed                          object
color                          object
intake_condition               object
intake_type                    object
sex_upon_intake                object
count                           int64
age_upon_intake_(days)          int64
age_upon_intake_(years)       float64
age_upon_intake_age_group      object
intake_datetime                object
intake_month                    int64
intake_year                     int64
intake_monthyear               object
intake_weekday                 object
intake_hour                     int64
intake_number                 float64
time_in_shelter                object
time_in_shelter_days          float64
age_upon_outcome               object
date_of_birth                  object
sex_upon_outcome               object
age_upon_out

In [302]:
df = pd.read_csv('train.csv')
df = df.drop(['animal_id_outcome',
#        'dob_month',
#        'animal_type',
        'age_upon_intake_(days)',
        'age_upon_intake_(years)',
        'intake_datetime',
        'intake_monthyear',
        'intake_hour',
        'sex_upon_intake',
        'date_of_birth',
        'age_upon_outcome',
        'age_upon_outcome_(days)',
        'time_in_shelter',
        'age_upon_outcome_(years)',
        'outcome_datetime',
        'outcome_number',
        'outcome_monthyear'], axis=1)

In [303]:
encoder = LabelEncoder()

df['sex_upon_outcome'] = encoder.fit_transform(df['sex_upon_outcome'].astype('str'))

df['animal_type'] = encoder.fit_transform(df['animal_type'].astype('str'))

df['breed'] = encoder.fit_transform(df['breed'].astype('str'))

df['color'] = encoder.fit_transform(df['color'].astype('str'))

df['age_upon_intake_age_group'] = encoder.fit_transform(df['intake_condition'].astype('str'))
df['sex_upon_outcome'] = encoder.fit_transform(df['sex_upon_outcome'].astype('str'))
df['age_upon_outcome_age_group'] = encoder.fit_transform(df['age_upon_outcome_age_group'].astype('str'))
df['outcome_weekday'] = encoder.fit_transform(df['outcome_weekday'].astype('str'))

df['outcome_type'] = encoder.fit_transform(df['outcome_type'].astype('str'))

df['intake_type'] = encoder.fit_transform(df['intake_type'].astype('str'))

df['intake_weekday'] = encoder.fit_transform(df['intake_weekday'].astype('str'))

df['intake_condition'] = encoder.fit_transform(df['intake_condition'].astype('str'))

def label_age_upon_intake(row):
    x = row['age_upon_intake'].split(' ')
    value = 0
    if x=='years':
        value = int(row['age_upon_intake'][:-6])*365
    elif x=='weeks':
        return int(row['age_upon_intake'][:-6])*7
    elif x=='months':
        return int(row['age_upon_intake'][:-7])*30
    elif x=='days':
        return int(row['age_upon_intake'][:-5])
    elif x=='year':
        return int(row['age_upon_intake'][:-5])*365
    elif x=='day':
        return int(row['age_upon_intake'][:-4])
    elif x=='month':
        return int(row['age_upon_intake'][:-6])*30
    elif x=='week':
        return int(row['age_upon_intake'][:-5])*7
    return value
    
df['age_upon_intake']=df.apply (lambda row: label_age_upon_intake (row),axis=1)



In [304]:
def float_to_int_intake(row):
    return int(row['intake_number'])
def float_to_int_outcome(row):
    return int(row['outcome_number'])
    
df['intake_number']=df.apply (lambda row: float_to_int_intake (row),axis=1)
#df['outcome_number']=df.apply (lambda row: float_to_int_outcome (row),axis=1)

In [305]:
def time_in_shelter_days(row):
    return int(row['time_in_shelter_days'])*365
    
df['time_in_shelter_days']=df.apply (lambda row: time_in_shelter_days (row),axis=1)

In [306]:
train, test = train_test_split(df, test_size=0.2)

y_train = train.outcome_type
x_train = train.drop(['outcome_type'],axis=1)
y_test = test.outcome_type
x_test = test.drop(['outcome_type'],axis=1)



In [307]:
lr = LogisticRegression()
lr.fit(x_train,y_train)
y_pred= lr.predict(x_test)



D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [309]:
rfm = RandomForestClassifier(n_estimators=70, oob_score=70, n_jobs=-1,
                            random_state=101,max_features=None, min_samples_leaf=30)
rfm.fit(x_train,y_train)
y_pred=rfm.predict(x_test)


In [311]:
dtree = DecisionTreeClassifier(max_depth=100, random_state=101,
                              max_features=None, min_samples_leaf=15)
dtree.fit(x_train,y_train)
y_pred=dtree.predict(x_test)

In [ ]:
svm = SVC(kernel="linear",C=0.025,random_state=101)
svm.fit(x_train,y_train)
y_pred=svm.predict(x_test)

In [ ]:
total = 0
correct = 0

for i in range(len(y_pred)):
    if y_pred[i]==y_test.iloc[i]:
        correct+=1
    total+=1
    
print(correct/total)

0.4509988494927309
